In [ ]:
# @title Copyright & License (click to expand)
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Monitoring for custom tabular models

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?name=Model%20Monitoring&download_url=https%3A%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fmodel_monitoring%2Fmodel_monitoring_custom.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Workbench AI Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/model_monitoring_custom.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring/model_monitoring_custom.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

This tutorial demonstrates how to use Vertex AI Model Monitoring for custom tabular models.

### Objective

In this notebook, you learn to use the `Vertex AI Model Monitoring` service to detect feature skew and drift in the input predict requests, for custom tabular models.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Model Monitoring`
- `Vertex AI Prediction`
- `Vertex AI Model` resource
- `Vertex AI Endpoint` resource

The steps performed include:

- Download a pre-trained custom tabular model.
- Upload the pre-trained model as a `Model` resource.
- Deploy the `Model` resource to the `Endpoint` resource.
- Configure the `Endpoint` resource for model monitoring.
- Generate synthetic prediction requests for skew.
- Generate synthetic prediction requests for drift.
- Interpret the monitored data.

Learn more about [Introduction to Vertex AI Model Monitoring](https://cloud.google.com/vertex-ai/docs/model-monitoring/overview).

### Model

This tutorial uses a pre-trained model, where the model artifacts are stored in a public Cloud Storage bucket. 

The model is based on [the blog post](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml). The idea behind this model is that your company has extensive log data describing how your game users have interacted with the site. The raw data contains the following categories of information:

- identity - unique player identitity numbers
- demographic features - information about the player, such as the geographic region in which a player is located
- behavioral features - counts of the number of times a  player has triggered certain game events, such as reaching a new level
- churn propensity - this is the label or target feature, it provides an estimated probability that this player will churn, i.e. stop being an active player.

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* BigQuery

Learn about [Vertext AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Before you begin

### Set up your local development environment

**If you are using Colab or Vertex AI Workbench notebooks**, your environment already meets
all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook dashboard.

## Installation

Install the packages required for executing this notebook.

In [ ]:
import os
import sys

assert sys.version_info.major == 3, "This notebook requires Python 3."

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install required packages.
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-aiplatform
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-bigquery

### Restart the kernel

After you install the SDK, you need to restart the notebook kernel so it can find the packages. You can restart kernel from *Kernel -> Restart Kernel*, or running the following:

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID or PROJECT_ID == "[your-project-id]" is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations throughout the rest of this notebook.  Below are regions supported for Vertex AI.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

**For this notebook, we recommend that you leave the region set to the default value us-central1**.

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

#### User Email

Set your user email address to receive monitoring alerts.

In [ ]:
USER_EMAIL = "[your-email-address]"  # @param {type:"string"}

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench notebooks**, your environment is already
authenticated.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "Vertex AI"
into the filter box, and select
   **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click **Create**. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Notes about service account and permission

**By default no configuration is required**, if you run into any permission related issue, please make sure the service accounts above have the required roles:

|Service account email|Description|Roles|
|---|---|---|
|PROJECT_NUMBER-compute@developer.gserviceaccount.com|Compute Engine default service account|Dataflow Admin, Dataflow Worker, Storage Admin, BigQuery Admin, Vertex AI User|
|service-PROJECT_NUMBER@gcp-sa-aiplatform.iam.gserviceaccount.com|AI Platform Service Agent|Vertex AI Service Agent|


1. Goto https://console.cloud.google.com/iam-admin/iam.
2. Check the "Include Google-provided role grants" checkbox.
3. Find the above emails.
4. Grant the corresponding roles.

### Using data source from a different project
- For the BQ data source, grant both service accounts the "BigQuery Data Viewer" role.
- For the CSV data source, grant both service accounts the "Storage Object Viewer" role.

### Import libraries

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud.aiplatform import model_monitoring
from google.cloud import bigquery

import matplotlib.pyplot as plt

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

### Create BigQuery client

Create the BigQuery client.

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

#### Set hardware accelerators

You can set hardware accelerators for prediction.

Set the variables `DEPLOY_GPU/DEPLOY_NGPU` to use a container image supporting a GPU and the number of GPUs allocated to the virtual machine (VM) instance. For example, to use a GPU container image with 4 Nvidia Tesla K80 GPUs allocated to each VM, you would specify:

    (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

See the [locations where accelerators are available](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators).

Otherwise specify `(None, None)` to use a container image to run on a CPU.

In [ ]:
DEPLOY_GPU, DEPLOY_NGPU = (aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80, 1)

#### Set pre-built containers

Set the pre-built Docker container image for prediction.

For the latest list, see [Pre-built containers for prediction](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers).

In [ ]:
DEPLOY_VERSION = "tf2-gpu.2-5"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### Set machine types

Next, set the machine types to use for training and prediction.

- Set the variable `DEPLOY_COMPUTE` to configure your compute resources for prediction.
 - `machine type`
     - `n1-standard`: 3.75GB of memory per vCPU
     - `n1-highmem`: 6.5GB of memory per vCPU
     - `n1-highcpu`: 0.9 GB of memory per vCPU
 - `vCPUs`: number of \[2, 4, 8, 16, 32, 64, 96 \]

*Note: You may also use n2 and e2 machine types for training and deployment, but they do not support GPUs*.

In [ ]:
MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

## Introduction to Vertex AI Model Monitoring

Vertex AI Model Monitoring is supported for AutoML tabular models and custom tabular model. You can monitor for skew and drift detection in the inbound prediction requests or skew and drift detection of the feature attribution in the outbound prediction response.

The following are the basic steps to enable model monitoring:

1. Deploy a `Vertex AI` tabular model to an `Vertex AI Endpoint`.
2. Configure a model monitoring specification.
3. Configure the alerts.
4. Upload the model monitoring specification to the `Vertex AI Endpoint`.

Once configured, you can enable/disable monitoring, change alerts and update the model monitoring configuration. 

When model monitoring is enabled, incoming prediction requests are logged in a BigQuery table. The input feature values contained in the logged requests are then analyzed for skew or drift. You can set a sampling rate to monitor a subset of the production inputs to a model.

The model monitoring service needs to know how to parse the feature values, which is referred to as the input schema. For AutoML tabular models, the input schema is automatically provided. For custom tabular models, the service will attempt to automatically derive the input schema from the first 1000 prediction requests. Alternatively, one can upload the input schema.

For skew detection, requires a baseline for the statistical distribution of values in the training data. For AutoML tabular models this is automatically derived. For custom tabular models, you upload the training data to the service, and have the service automatically derived the distribution.

### Upload the model artifacts as a `Vertex AI Model` resource

First, you upload the pre-trained custom tabular model artifacts as a `Vertex AI Model` resource using the `upload()` method, with the following parameters:

- `display_name`: The human readable name for the `Model` resource.
- `artifact_uri`: The Cloud Storage location of the model artifacts.
- `serving_container_image`: The serving container image to use when the model is deployed to a `Vertex AI Endpoint` resource.
- `sync`: Whether to wait for the process to complete, or return immediately (async).

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"

model = aiplatform.Model.upload(
    display_name="churn_" + UUID,
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True
)

print(model)

### Deploy the `Vertex AI Model` resource to a `Vertex AI Endpoint` resource

Next, you deploy your `Vertex AI Model` resource to a `Vertex AI Endpoint` resource using the `deploy()` method, with the following parameters:

- `deploy_model_display`: The human reable name for the deployed model.
- `machine_type`: The machine type for each VM node instance.
- `min_replica_count`: The minimum number of nodes to provision for auto-scaling.
- `max_replica_count`: The maximum number of nodes to provision for auto-scaling.
- `accelerator_type`: The type, if any, of GPU accelators per provisioned node.
- `accelrator_count`: The number, if any, of GPU accelators per provisioned node.

In [ ]:
MIN_NODES=1
MAX_NODES=1

endpoint = model.deploy(
    deployed_model_display_name="churn_" + UUID,
    machine_type=DEPLOY_COMPUTE,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    accelerator_type=DEPLOY_GPU.name,
    accelerator_count=DEPLOY_NGPU
)

## Configure a monitoring job

Configuring the monitoring job consists of the following specifications:

- `alert_config`: The email address(es) to send monitoring alerts to.
- `schedule_config`: The time window to analyze predictions.
- `logging_sampling_strategy`: The rate for sampling prediction requests. 
- `drift_config`: The features and drift thresholds to monitor.
- `skew_config`: The features and skew thresholds to monitor.

### Configure the alerting specification

First, you configure the `alerting_config` specification with the following settings:

- `user_emails`: A list of one or more email to send alerts to.
- `enable_logging`: Streams detected anomalies to Cloud Logging. Default is False.

In [ ]:
# Create alerting configuration.
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=[USER_EMAIL], 
    enable_logging=True
)

### Configure the monitoring interval specification

Next, you configure the `schedule_config` specification with the following settings:

- `monitor_interval`:  Sets the model monitoring job scheduling interval in hours. Minimum time interval is 1 hour.

In [ ]:
# Monitoring Interval 
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(
    monitor_interval=MONITOR_INTERVAL
)

### Configure the sampling specification

Next, you configure the `logging_sampling_strategy` specification with the following settings:

- `sample_rate`: The rate as a percentage (between 0 and 1) to randomly sample predictions for monitoring. Select samples are logged to a BigQuery table.


In [ ]:
# Sampling rate (optional, default=.8)
SAMPLE_RATE = 0.5  # @param {type:"number"}

# Create sampling configuration
logging_sampling_strategy = model_monitoring.RandomSampleConfig(
    sample_rate=SAMPLE_RATE
)

### Configure the drift detection specification

Next, you configure the `drift_config` specification with the following settings:

- `drift_thresholds`: A dictionary of key/value pairs where the keys are the input features for monitor for drift. The value is the detection threshold. When not specified, the default drift threshold for a feature is 0.3 (30%).

*Note:* Enabling drift detection is optional.

In [ ]:
DEFAULT_THRESHOLD_VALUE = 0.05

DRIFT_THRESHOLDS = {
    "country": DEFAULT_THRESHOLD_VALUE,
    "cnt_user_engagement": DEFAULT_THRESHOLD_VALUE,
}

drift_config = model_monitoring.DriftDetectionConfig(
    drift_thresholds=DRIFT_THRESHOLDS
)


### Configure the skew detection specification

Next, you configure the `skew_config` specification with the following settings:

- `data_source`: The source of the dataset of the original training data. The format of the source defaults to a BigQuery table. Otherwise the setting `data_format` must be set to one of the values below. The location of the data must be a Cloud Storage location.
  - `csv`: 
  - `jsonl`:
  - `tf-record`:
- `skew_thresholds`: A dictionary of key/value pairs where the keys are the input features for monitor for skew. The value is the detection threshold. When not specified, the default skew threshold for a feature is 0.3 (30%).
- `target_field`: The target label for the training dataset

*Note:* Enabling skew detection is optional.

In [ ]:
# URI to training dataset.
DATASET_BQ_URI = "bq://mco-mm.bqmlga4.train"  # @param {type:"string"}
# Prediction target column name in training dataset.
TARGET = "churned"

SKEW_THRESHOLDS = {
    "country": DEFAULT_THRESHOLD_VALUE,
    "cnt_user_engagement": DEFAULT_THRESHOLD_VALUE,
}

skew_config = model_monitoring.SkewDetectionConfig(
    data_source=DATASET_BQ_URI,
    skew_thresholds=SKEW_THRESHOLDS,
    target_field=TARGET
)

### Assemble the objective specification

Finally, you assemble the objective specification `objective_config` with the following settings:

- `skew_detection_config`: (Optional) The specification for the skew detection configuration.
- `drift_detection_config`: (Optional) The specification for the drift detection configuration.
- `explanation_config`: (Optional) The specification for explanations when enabling monitoring for feature attributions.

In [ ]:
objective_config = model_monitoring.ObjectiveConfig(
    skew_detection_config=skew_config,
    drift_detection_config=drift_config,
    explanation_config=None
)

### Create the monitoring job

You create a monitoring job, with your monitoring specifications, using the `aiplatform.ModelDeploymentMonitoringJob.create()` method, with the following parameters:

- `display_name`: The human readable name for the monitoring job.
- `project`: The project ID.
- `region`: The region.
- `endpoint`: The fully qualified resource name of the `Vertex AI Endpoint` to enable monitoring.
- `logging_sampling_strategy`: The specification for the sampling configuration.
- `schedule_config`: The specification for the scheduling configuration.
- `alert_config`: The specification for the alerting configuration.
- `objective_configs`: The specification for the objectives configuration.

In [ ]:
monitoring_job = aiplatform.ModelDeploymentMonitoringJob.create(
    display_name="churn_" + UUID,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
    logging_sampling_strategy=random_sampling,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
)

print(monitoring_job.gca_resource)

#### Wait for the notification of the monitoring job.

An email notification is sent to the email address in the alerting configuration, notifying that the model monitoring job is now enabled.

Before proceeding, wait for the email notification. The contents will appear like:

<blockquote>
Hello Vertex AI Customer,

You are receiving this mail because you are using the Vertex AI Model Monitoring service.
This mail is to inform you that we received your request to set up drift or skew detection for the Prediction Endpoint listed below. Starting from now, incoming prediction requests will be sampled and logged for analysis.
Raw requests and responses will be collected from prediction service and saved in bq://[your-project-id].model_deployment_monitoring_[endpoint-id].serving_predict .
</blockquote>

#### The BigQuery dataset and table for monitoring

The BigQuery dataset and table for logging the sampled requests are respectively named `model_deployment_monitoring_<ENDPOINT_ID>`, where <ENDPOINT_ID> is the numerical identifier for the `Vertex AI Endpoint` resource, and `serving_predict`.

In [ ]:
if os.getenv("IS_TESTING"):
    import time
    time.sleep(120)

## Initialize the parsing for automatically generating the input schema

After your `Endpoint` receives a 1000 prediction requests, the modeling service will automatically parse and create the `input schema`.

### Create the 1000 instance data

In this example, the first 1000 entries in the BigQuery training data are used as the first 1000 prediction requests. 

*Note:* In this context, each instance is a prediction request. In otherwords, sending 1000 prediction requests of a single instance is the same as sending a single prediction request with 1000 instances.

In [ ]:
# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(
    table,
    max_results=1000
)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value == None:
            value = ''
        instance[key] = value
    instances.append(instance)
    
print(len(instances))

### Make the initial prediction request

Next, you send the the 1000 prediction request to your `Vertex AI Endpoint` resource using the `predict()` method.

In [ ]:
response = endpoint.predict(instances=instances)

prediction = response[0]

# print the prediction for the first instance
print(prediction[0])

### Pause the monitoring job

You can pause and resume a monitoring job with the methods `pause()` and `resume()`, respectively.

In [ ]:
monitoring_job.pause()
monitoring_job.resume()

### Delete the monitoring job

You can delete the monitoring job using the `delete()` method.

In [ ]:
monitoring_job.delete()

### Delete the logged sampled data

In [ ]:
# Delete the temporary BigQuery dataset
ENDPOINT_ID = endpoint.resource_name.split('/')[-1]

! bq rm -r -f {PROJECT_ID}:model_deployment_monitoring_{ENDPOINT_ID}

#### Undeploy and delete the `Vertex AI Endpoint` resource

Your `Vertex AI Endpoint` resource can be deleted using the `delete()` method. Prior to deleting, any model deployed to your `Vertex AI Endpoint` resource, must first be undeployed.

In [ ]:
endpoint.undeploy_all()
endpoint.delete()

#### Delete the `Vertex AI Model` resource

Your `Vertex AI Model` resource can be deleted using the `delete()` method.

In [ ]:
model.delete()

### Cleanup